In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import lazypredict
from lazypredict.Supervised import LazyClassifier
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

C:\Users\berna\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\berna\anaconda3\envs\teachopencadd\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
# Set path to this notebook
HERE = Path(_dh[-1])
HDAC1and6 = Path(HERE).resolve().parents[1]/'input'
output = HERE/'OUTPUT'

In [3]:
df1 = pd.read_pickle(HDAC1and6/"HDAC1and6_1024B.csv")
df1 = df1.sample(frac=1).reset_index(drop=True)
df1.shape

(1339, 6)

In [4]:
# Add column for selectivity
df1["selectivity"] = np.zeros(len(df1))

# Mark every molecule as selective if SelectivityWindow is >=2 or >=-2, 0 otherwise
df1.loc[df1[df1.SelectivityWindow >= 2.0].index, "selectivity"] = 1.0
df1.loc[df1[df1.SelectivityWindow <= -2.0].index, "selectivity"] = 1.0
# NBVAL_CHECK_OUTPUT
print("Number of selective compounds:", int(df1.selectivity.sum()))
print("Number of nonselective compounds:", len(df1) - int(df1.selectivity.sum()))

Number of selective compounds: 334
Number of nonselective compounds: 1005


In [5]:
df1.head(5)

,molecule_chembl_id,fp_MACCS,fp_Morgan3,fp_MorganF,fp_MAP4,SelectivityWindow,selectivity
0,CHEMBL468513,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[12350052, 10511, 2475943, 2183376, 10851153, ...",0.04,0.00
1,CHEMBL3771288,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...","[1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...","[14716226, 11837907, 9564045, 218190, 6876230,...",0.05,0.00
2,CHEMBL461142,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...","[11575212, 736579, 11418700, 15887816, 278633,...",0.67,0.00
3,CHEMBL2347007,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[1793055, 23287459, 9798839, 6338606, 5280553,...",-0.04,0.00
4,CHEMBL4284448,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ...","[6411541, 3821889, 137380, 1561872, 2471491, 1...",2.31,1.00


#### By using MACCS fingerprints that are 166 bits 

In [6]:
X1 = np.array(list((df1['fp_MACCS']))).astype(float)
#X.shape
Y1 = df1["selectivity"].values
#y.shape

In [7]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X1, Y1, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train1.shape)
print("Shape of test data:", X_test1.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (1071, 167)
Shape of test data: (268, 167)


In [8]:
# Defines and builds the LazyClassifier
clf1 =  LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
model1,predictions1 = clf1.fit(X_train1, X_test1, Y_train1, Y_test1)

 93%|████████████████████████████████████████████████████████████████████████████▎     | 27/29 [00:03<00:00,  9.11it/s]

[19:38:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:04<00:00,  6.41it/s]


In [9]:
predictions1.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.90,0.85,0.85,0.90,0.19
BaggingClassifier,0.90,0.84,0.84,0.90,0.13
RandomForestClassifier,0.90,0.84,0.84,0.90,0.28
XGBClassifier,0.90,0.84,0.84,0.90,0.46
ExtraTreesClassifier,0.90,0.84,0.84,0.90,0.25
KNeighborsClassifier,0.89,0.82,0.82,0.89,0.14
SVC,0.89,0.80,0.80,0.88,0.23
DecisionTreeClassifier,0.88,0.82,0.82,0.88,0.04
ExtraTreeClassifier,0.87,0.81,0.81,0.87,0.02


### USING 1024 BITS FINGERPRINTS

In [10]:
#By using Morgan fingerprints with radius of 3 and 1024 bits
X2 = np.array(list((df1['fp_Morgan3']))).astype(float)
#X.shape
Y2 = df1["selectivity"].values
#y.shape

In [11]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X2, Y2, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train2.shape)
print("Shape of test data:", X_test2.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (1071, 1024)
Shape of test data: (268, 1024)


In [12]:
clf2 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models2, predictions2 = clf2.fit(X_train2, X_test2, Y_train2, Y_test2)

 90%|█████████████████████████████████████████████████████████████████████████▌        | 26/29 [00:16<00:02,  1.35it/s]

[19:39:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:18<00:00,  1.55it/s]


In [13]:
predictions2.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.93,0.86,0.86,0.92,0.48
ExtraTreesClassifier,0.92,0.86,0.86,0.92,0.57
XGBClassifier,0.92,0.86,0.86,0.92,1.15
LGBMClassifier,0.91,0.84,0.84,0.91,0.57
KNeighborsClassifier,0.90,0.86,0.86,0.90,0.85
SVC,0.91,0.81,0.81,0.90,1.72
BaggingClassifier,0.90,0.81,0.81,0.90,0.81
AdaBoostClassifier,0.89,0.80,0.80,0.88,1.01
Perceptron,0.88,0.83,0.83,0.88,0.14


In [14]:
#By using Morgan fingerprints with radius of 3 and 1024 bits and feautures
X3 = np.array(list((df1['fp_MorganF']))).astype(float)
#X.shape
Y3 = df1["selectivity"].values
#y.shape

In [15]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train3, X_test3, Y_train3, Y_test3 = train_test_split(X3, Y3, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train3.shape)
print("Shape of test data:", X_test3.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (1071, 1024)
Shape of test data: (268, 1024)


In [16]:
clf3 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models3, predictions3 = clf3.fit(X_train3, X_test3, Y_train3, Y_test3)

 90%|█████████████████████████████████████████████████████████████████████████▌        | 26/29 [00:17<00:02,  1.38it/s]

[19:39:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:18<00:00,  1.53it/s]


In [17]:
predictions3.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
SVC,0.92,0.84,0.84,0.91,1.70
ExtraTreesClassifier,0.91,0.84,0.84,0.91,0.60
LGBMClassifier,0.91,0.84,0.84,0.91,0.63
XGBClassifier,0.90,0.84,0.84,0.90,1.16
RandomForestClassifier,0.90,0.84,0.84,0.90,0.52
BernoulliNB,0.90,0.85,0.85,0.90,0.09
PassiveAggressiveClassifier,0.90,0.85,0.85,0.90,0.15
BaggingClassifier,0.90,0.82,0.82,0.90,0.83
NearestCentroid,0.89,0.85,0.85,0.89,0.10


In [18]:
#By using MAP4 fingerprints with 1024 bits
X4 = np.array(list((df1['fp_MAP4']))).astype(float)
#X.shape
Y4 = df1["selectivity"].values
#y.shape

In [19]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train4, X_test4, Y_train4, Y_test4 = train_test_split(X4, Y4, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train4.shape)
print("Shape of test data:", X_test4.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (1071, 1024)
Shape of test data: (268, 1024)


In [20]:
# Defines and builds the lazyclassifier
clf4 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models4, predictions4 = clf4.fit(X_train4, X_test4, Y_train4, Y_test4)

 90%|█████████████████████████████████████████████████████████████████████████▌        | 26/29 [00:27<00:02,  1.37it/s]

[19:40:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:38<00:00,  1.34s/it]


In [21]:
predictions4.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
BaggingClassifier,0.92,0.84,0.84,0.91,6.06
RandomForestClassifier,0.91,0.81,0.81,0.91,1.84
LGBMClassifier,0.91,0.83,0.83,0.91,8.07
XGBClassifier,0.91,0.82,0.82,0.90,3.01
ExtraTreesClassifier,0.90,0.78,0.78,0.89,0.62
AdaBoostClassifier,0.89,0.80,0.80,0.89,4.99
KNeighborsClassifier,0.88,0.78,0.78,0.88,0.88
SVC,0.89,0.75,0.75,0.87,1.38
PassiveAggressiveClassifier,0.85,0.81,0.81,0.86,0.19


## USE 2048 BITS FINGERPRINTS

In [22]:
df2 = pd.read_pickle(HDAC1and6/"HDAC1and6_2048B.csv")
df2 = df2.sample(frac=1).reset_index(drop=True)
df2

,molecule_chembl_id,fp_Morgan3,fp_MorganF,fp_MAP4,SelectivityWindow
0,CHEMBL3774858,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[10379840, 20516887, 5934914, 7125875, 1044292...",0.62
1,CHEMBL2385955,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[5061059, 10511, 2475943, 131744, 2157755, 361...",0.32
2,CHEMBL4079007,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3446718, 4669681, 7998790, 17170035, 492964, ...",0.17
3,CHEMBL512644,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[2331191, 10511, 137380, 9391761, 7332227, 163...",0.82
4,CHEMBL2047675,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[33116081, 2349897, 1015231, 15008225, 193508,...",-0.40
...,...,...,...,...,...
1334,CHEMBL3356926,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[2171934, 2201708, 2475943, 207924, 3858537, 9...",-0.48
1335,CHEMBL3628322,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3300235, 1465270, 59082, 10483303, 4853714, 4...",-0.92
1336,CHEMBL2431901,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5976924, 7588115, 8033062, 3196706, 1293667, ...",3.09
1337,CHEMBL3098692,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[2770663, 2201708, 526260, 12740216, 2471491, ...",0.84


In [23]:
# Add column for selectivity
df2["selectivity"] = np.zeros(len(df2))

# Mark every molecule as selective if SelectivityWindow is >=2 or >=-2, 0 otherwise
df2.loc[df2[df2.SelectivityWindow >= 2.0].index, "selectivity"] = 1.0
df2.loc[df2[df2.SelectivityWindow <= -2.0].index, "selectivity"] = 1.0
# NBVAL_CHECK_OUTPUT
print("Number of selective compounds:", int(df2.selectivity.sum()))
print("Number of nonselective compounds:", len(df2) - int(df2.selectivity.sum()))

Number of selective compounds: 334
Number of nonselective compounds: 1005


In [24]:
#2048 bits Morgan fingerprints with radius of 3
X5 = np.array(list((df2['fp_Morgan3']))).astype(float)
#X.shape
Y5 = df2["selectivity"].values
#y.shape

In [25]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train5, X_test5, Y_train5, Y_test5 = train_test_split(X5, Y5, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train5.shape)
print("Shape of test data:", X_test5.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (1071, 2048)
Shape of test data: (268, 2048)


In [26]:
clf5 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models5, predictions5 = clf5.fit(X_train5, X_test5, Y_train5, Y_test5)

 90%|█████████████████████████████████████████████████████████████████████████▌        | 26/29 [00:41<00:04,  1.46s/it]

[19:40:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:46<00:00,  1.59s/it]


In [27]:
predictions5.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.92,0.87,0.87,0.92,1.05
RandomForestClassifier,0.91,0.84,0.84,0.91,0.77
XGBClassifier,0.89,0.83,0.83,0.89,3.07
KNeighborsClassifier,0.88,0.81,0.81,0.88,1.71
SVC,0.89,0.77,0.77,0.88,3.57
BaggingClassifier,0.88,0.80,0.80,0.88,1.96
LGBMClassifier,0.87,0.80,0.80,0.87,1.42
NearestCentroid,0.87,0.80,0.80,0.87,0.35
PassiveAggressiveClassifier,0.86,0.82,0.82,0.86,0.32


In [28]:
#2048 bits Morgan fingerprints with feautures
X6 = np.array(list((df2['fp_MorganF']))).astype(float)
#X.shape
Y6 = df2["selectivity"].values
#y.shape

In [29]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train6, X_test6, Y_train6, Y_test6 = train_test_split(X6, Y6, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train6.shape)
print("Shape of test data:", X_test6.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (1071, 2048)
Shape of test data: (268, 2048)


In [30]:
clf6 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models6, predictions6 = clf6.fit(X_train6, X_test6, Y_train6, Y_test6)

 90%|█████████████████████████████████████████████████████████████████████████▌        | 26/29 [00:38<00:04,  1.39s/it]

[19:41:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:42<00:00,  1.47s/it]


In [31]:
predictions6.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.91,0.87,0.87,0.91,0.77
ExtraTreesClassifier,0.90,0.85,0.85,0.90,1.05
XGBClassifier,0.90,0.84,0.84,0.90,3.07
LGBMClassifier,0.89,0.85,0.85,0.89,1.24
BernoulliNB,0.89,0.85,0.85,0.89,0.17
BaggingClassifier,0.89,0.83,0.83,0.89,1.77
NearestCentroid,0.88,0.81,0.81,0.88,0.20
DecisionTreeClassifier,0.87,0.81,0.81,0.87,0.59
ExtraTreeClassifier,0.84,0.79,0.79,0.85,0.17


In [32]:
#2048 bits MAP4 fingerprints
X7 = np.array(list((df2['fp_MAP4']))).astype(float)
#X.shape
Y7 = df2["selectivity"].values
#y.shape

In [33]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train7, X_test7, Y_train7, Y_test7 = train_test_split(X7, Y7, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train7.shape)
print("Shape of test data:", X_test7.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (1071, 2048)
Shape of test data: (268, 2048)


In [34]:
# Defines and builds the lazyclassifier
clf7 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models7, predictions7 = clf7.fit(X_train7, X_test7, Y_train7, Y_test7)

 90%|█████████████████████████████████████████████████████████████████████████▌        | 26/29 [00:56<00:04,  1.41s/it]

[19:42:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [01:19<00:00,  2.74s/it]


In [35]:
predictions7.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.93,0.85,0.85,0.92,17.45
RandomForestClassifier,0.91,0.80,0.80,0.90,3.28
XGBClassifier,0.90,0.80,0.80,0.89,5.44
Perceptron,0.88,0.82,0.82,0.88,0.21
DecisionTreeClassifier,0.88,0.81,0.81,0.88,2.33
BaggingClassifier,0.89,0.76,0.76,0.88,12.07
ExtraTreesClassifier,0.88,0.76,0.76,0.87,0.93
SGDClassifier,0.87,0.75,0.75,0.86,0.23
AdaBoostClassifier,0.86,0.76,0.76,0.86,8.41


In [36]:
with pd.ExcelWriter(output/"LazyPredictResults.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet
    predictions1.to_excel(writer, sheet_name="MACCS", )
    predictions2.to_excel(writer, sheet_name="Morgan 1024 Bits", )
    predictions3.to_excel(writer, sheet_name="MorganF 1024 Bits", )
    predictions4.to_excel(writer, sheet_name="MAP4 1024 Bits", )
    predictions5.to_excel(writer, sheet_name="Morgan 2048 Bits", )
    predictions6.to_excel(writer, sheet_name="MorganF 2048 Bits", )
    predictions7.to_excel(writer, sheet_name="MAP4 2048 Bits", )
    